# Quantitative validation of candidates

In [ ]:
import pandas as pd
import itertools as it
import pymysql
import numpy as np
import datetime
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()
import MySQLdb
import networkx as nx
import pymysql
import xml.etree.ElementTree as ET
from tqdm import tqdm
import requests
import json
import time
import csv
import re
import os
import warnings
import torch
import pickle
from node2vec import Node2Vec

In [ ]:
# First, you must construct uspto, disgenet, and dgidb for your environment
# Change below codes for your environment

import os
import pymysql

host = os.getenv("DB_HOST")
port = int(os.getenv("DB_PORT"))
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
db_name = os.getenv("DB_NAME")
charset = os.getenv("DB_CHARSET", "utf8mb4")

conn = pymysql.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    db=db_name,
    charset=charset,
)
cursor = conn.cursor(pymysql.cursors.DictCursor)


In [3]:
cursor.execute('''SELECT p.reg_num, p.type, p.lifetime
FROM patent p
WHERE p.reg_num IN (
SELECT distinct i.reg_num
FROM ipcr i
WHERE i.ipc4 = 'A61K' AND i.main_group <> '8'
);''')
data = cursor.fetchall()

In [4]:
pharma_pat = pd.DataFrame(data)

In [5]:
pharma_pat

,reg_num,type,lifetime
0,9138474,utility,0.0
1,8835433,utility,0.0
2,6113943,utility,20.0
3,10653639,utility,NaN
4,10143741,utility,0.0
...,...,...,...
274248,6323187,utility,12.0
274249,4747868,utility,4.0
274250,7858795,utility,0.0
274251,4992463,utility,20.0


In [5]:
pharma_pat

,reg_num,type,lifetime
0,9138474,utility,0.0
1,8835433,utility,0.0
2,6113943,utility,20.0
3,10653639,utility,NaN
4,10143741,utility,0.0
...,...,...,...
274248,6323187,utility,12.0
274249,4747868,utility,4.0
274250,7858795,utility,0.0
274251,4992463,utility,20.0


In [6]:
pharma_pat = pharma_pat[pharma_pat['type'] == 'utility']

In [7]:
pharma_pat = pharma_pat[pharma_pat['lifetime'] > 1]

In [8]:
pharma_pat

,reg_num,type,lifetime
2,6113943,utility,20.0
8,4491591,utility,4.0
14,8895072,utility,4.0
15,6402733,utility,8.0
20,7563459,utility,20.0
...,...,...,...
274245,4496560,utility,4.0
274247,6235735,utility,12.0
274248,6323187,utility,12.0
274249,4747868,utility,4.0


In [9]:
# For this data, you can visit https://www.fda.gov/drugs/drug-approvals-and-databases/orange-book-data-files

fda_pat = pd.read_csv("patent.txt", sep="~")

In [10]:
fda_pat = fda_pat[['Patent_No']]

In [11]:
fda_pat['reg_num'] = fda_pat['Patent_No'] 

In [12]:
del fda_pat['Patent_No'] 

In [13]:
mask = pharma_pat['reg_num'].isin(fda_pat['reg_num'])

In [14]:
pharma_pat = pharma_pat[~mask]

In [15]:
del pharma_pat['type']

In [16]:
pharma_pat

,reg_num,lifetime
2,6113943,20.0
8,4491591,4.0
14,8895072,4.0
15,6402733,8.0
20,7563459,20.0
...,...,...
274245,4496560,4.0
274247,6235735,12.0
274248,6323187,12.0
274249,4747868,4.0


In [17]:
fda_pat

,reg_num
0,7625884
1,7560445
2,8455524
3,8455524
4,7422388
...,...
16745,7378508*PED
16746,7906489*PED
16747,7863249*PED
16748,8859510*PED


In [18]:
cursor.execute('''SELECT distinct a.reg_num, a.app_date, p.Patent_Expire_Date, DATEDIFF(p.Patent_Expire_Date, a.app_date)/365 lifetime
FROM patent.application a
inner JOIN db_name.fda_patents p
WHERE a.reg_num = p.Patent_No;''')
data = cursor.fetchall()

In [19]:
fda_pat_lifetime = pd.DataFrame(data)

In [41]:
fda_pat_lifetime['lifetime'] = fda_pat_lifetime['lifetime'].astype(float)

In [43]:
fda_pat_lifetime

,reg_num,app_date,Patent_Expire_Date,lifetime
0,7625884,2007-10-23,2026-08-24,18.8493
1,7560445,2006-06-30,2027-02-01,20.6055
2,8455524,2010-12-17,2027-04-18,16.3452
3,7422388,2007-04-25,2027-04-25,20.0137
4,7192938,2004-11-29,2023-05-06,18.4438
...,...,...,...,...
4909,8147867,2005-05-02,2028-08-29,23.3425
4910,7776007,2004-12-06,2026-11-22,21.9753
4911,10669236,2018-09-07,2038-09-07,20.0137
4912,9446057,2015-11-12,2034-12-23,19.1260


In [44]:
import pandas as pd
from scipy import stats

# assuming you have two dataframes, pharma_pat and fda_pat_lifetime, and 'column_name' is the column you're interested in

# extract the column of interest
pharma_pat_lt = pharma_pat['lifetime']
fda_pat_lt = fda_pat_lifetime['lifetime']

# perform the t-test
t_statistic, p_value = stats.ttest_ind(pharma_pat_lt, fda_pat_lt)

print(f"t-statistic: {t_statistic}, p-value: {p_value}")

t-statistic: -55.140382066144745, p-value: 0.0


In [45]:
from scipy import stats

# Using the previous variables:
# pharma_pat_lt and fda_pat_lt

# perform Levene's test
w_statistic, p_value_levene = stats.levene(pharma_pat_lt, fda_pat_lt)

print(f"W-statistic from Levene's test: {w_statistic}, p-value: {p_value_levene}")

W-statistic from Levene's test: 2632.1581337286284, p-value: 0.0


In [46]:
from scipy import stats

# assuming you have already extracted the column of interest
# pharma_pat_lt and fda_pat_lt

# perform the Welch's t-test
t_statistic_welch, p_value_welch = stats.ttest_ind(pharma_pat_lt, fda_pat_lt, equal_var=False)

print(f"Welch's t-statistic: {t_statistic_welch}, p-value: {p_value_welch}")

Welch's t-statistic: -82.58137582722809, p-value: 0.0


In [ ]:
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# Shapiro-Wilk test for pharma_pat_lt
shapiro_stat_p, shapiro_p_p = stats.shapiro(pharma_pat_lt)

# Shapiro-Wilk test for fda_pat_lt
shapiro_stat_f, shapiro_p_f = stats.shapiro(fda_pat_lt)

print(f"Shapiro-Wilk for pharma_pat_lt: statistic = {shapiro_stat_p}, p-value = {shapiro_p_p}")
print(f"Shapiro-Wilk for fda_pat_lt: statistic = {shapiro_stat_f}, p-value = {shapiro_p_f}")

In [ ]:
# Shapiro-Wilk for pharma_pat_lt: statistic = 0.8110537528991699, p-value = 0.0
# Shapiro-Wilk for fda_pat_lt: statistic = 0.9778901934623718, p-value = 5.510794698297405e-27

In [49]:
dr_candidates = pd.read_csv('dr_candidates.csv')

In [51]:
final_candidates = pd.read_csv('clinical_data.csv')

In [115]:
candidates = pd.merge(dr_candidates, final_candidates, on='drug_concept_id')

In [116]:
del candidates['Unnamed: 0_x']

In [117]:
del candidates['disease_x']

In [118]:
del candidates['Unnamed: 0_y']

In [119]:
del candidates['rank_y']

In [62]:
del candidates['gene_count_rank']

In [120]:
del candidates['clinicaltrialsgov']

In [121]:
del candidates['pubmed_y']

In [122]:
del candidates['disease_y']

In [123]:
del candidates['clinical_sums']

In [124]:
del candidates['drug_name_y']

In [125]:
del candidates['Unnamed: 12']

In [126]:
del candidates['Unnamed: 13']

In [127]:
del candidates['Unnamed: 14']

In [72]:
del candidates['cosine_similarity_x']

In [73]:
del candidates['cosine_similarity_y']

In [74]:
del candidates['gene_count']

In [128]:
candidates

,drug_concept_id,cosine_similarity_x,rank_x,clinical_trials,pubmed_x,clinical_sum,drug_name_x,cosine_similarity_y,gene_count,gene_count_rank,final_rank
0,CHEMBL254836,0.918763,1.0,0.0,0.0,0.0,PT-315,-0.164230,0,209,229
1,CHEMBL4298172,0.864723,2.0,0.0,0.0,0.0,Soticlestat,0.668359,1,62,74
2,CHEMBL225411,0.821627,3.0,0.0,0.0,0.0,GW-842166X,0.655229,1,62,89
3,CHEMBL3527358,0.808408,4.0,0.0,0.0,0.0,Metapro,0.533799,1,62,199
4,CHEMBL4297504,0.807645,5.0,0.0,0.0,0.0,Beloranib,0.573598,1,62,180
...,...,...,...,...,...,...,...,...,...,...,...
224,CHEMBL357076,0.701239,225.0,0.0,0.0,0.0,CompB,0.594741,1,62,168
225,CHEMBL2311194,0.701226,226.0,0.0,0.0,0.0,(+)-Pentazocine,0.624281,1,62,132
226,CHEMBL185515,0.701110,227.0,0.0,0.0,0.0,A-317567,0.682624,1,62,63
227,CHEMBL1677,0.700845,228.0,0.0,2.0,2.0,Tacrine hydrochloride,0.621830,2,24,106


In [112]:
candidates

,drug_concept_id,rank_x,clinical_trials,pubmed_x,clinical_sum,drug_name_x,final_rank,const
0,CHEMBL254836,1.0,0.0,0.0,0.0,PT-315,229,1
1,CHEMBL4298172,2.0,0.0,0.0,0.0,Soticlestat,74,1
2,CHEMBL225411,3.0,0.0,0.0,0.0,GW-842166X,89,1
3,CHEMBL3527358,4.0,0.0,0.0,0.0,Metapro,199,1
4,CHEMBL4297504,5.0,0.0,0.0,0.0,Beloranib,180,1
...,...,...,...,...,...,...,...,...
224,CHEMBL357076,225.0,0.0,0.0,0.0,CompB,168,1
225,CHEMBL2311194,226.0,0.0,0.0,0.0,(+)-Pentazocine,132,1
226,CHEMBL185515,227.0,0.0,0.0,0.0,A-317567,63,1
227,CHEMBL1677,228.0,0.0,2.0,2.0,Tacrine hydrochloride,106,1


In [133]:
import pandas as pd

# Min-Max Scaling
candidates['sim_x_min_max_scaled'] = (candidates['cosine_similarity_x'] - candidates['cosine_similarity_x'].min()) / (candidates['cosine_similarity_x'].max() - candidates['cosine_similarity_x'].min())

In [134]:
import pandas as pd

# Min-Max Scaling
candidates['sim_y_min_max_scaled'] = (candidates['cosine_similarity_y'] - candidates['cosine_similarity_y'].min()) / (candidates['cosine_similarity_y'].max() - candidates['cosine_similarity_y'].min())

In [135]:
# Wilcoxon Signed-Rank Test
w_stat, p_value_wilcoxon = wilcoxon(candidates['sim_x_min_max_scaled'], candidates['sim_y_min_max_scaled'])
print(f"Wilcoxon signed-rank test: W-statistic: {w_stat}, p-value: {p_value_wilcoxon:.5f}")

Wilcoxon signed-rank test: W-statistic: 236.0, p-value: 0.00000
